<a href="https://colab.research.google.com/github/SunghoonSeok/Study/blob/master/%EC%99%B8%EB%B6%80_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%A1%9C%EC%BB%AC_%ED%8C%8C%EC%9D%BC%2C_%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B8%8C%2C_%EC%8A%A4%ED%94%84%EB%A0%88%EB%93%9C%EC%8B%9C%ED%8A%B8%2C_Cloud_Storage%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 노트북에서는 외부 소스의 데이터를 로드 및 저장하는 레시피를 이용할 수 있습니다.

In [ ]:
from google.colab import files
myfile = files.upload()



Saving KODEX 코스닥150 선물인버스.csv to KODEX 코스닥150 선물인버스.csv
Saving 삼성전자.csv to 삼성전자.csv
Saving 삼성전자0115.csv to 삼성전자0115.csv


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import io
# from pandas import read_csv
# df = read_csv('c:/data/test/삼성전자.csv', index_col=0, header=0)
df = pd.read_csv(io.BytesIO(myfile['삼성전자.csv']), index_col=0, header=0, encoding='cp949')


print(df.shape) # (2400, 14)
#Index(['시가', '고가', '저가', '종가', '등락률', '거래량', '금액(백만)', '신용비', '개인', '기관',
#       '외인(수량)', '외국계', '프로그램', '외인비']

# str -> int, 불필요한 행 제거
df = df.drop(['2018-05-03','2018-05-02','2018-04-30'])
df['시가'] =df['시가'].str.replace(',','').astype('int64')
df['고가'] =df['고가'].str.replace(',','').astype('int64')
df['저가'] =df['저가'].str.replace(',','').astype('int64')
df['종가'] =df['종가'].str.replace(',','').astype('int64')
df['거래량'] =df['거래량'].str.replace(',','').astype('int64')
df['금액(백만)'] =df['금액(백만)'].str.replace(',','').astype('int64')
df['개인'] =df['개인'].str.replace(',','').astype('int64')
df['기관'] =df['기관'].str.replace(',','').astype('int64')
df['외인(수량)'] =df['외인(수량)'].str.replace(',','').astype('int64')
df['외국계'] =df['외국계'].str.replace(',','').astype('int64')
df['프로그램'] =df['프로그램'].str.replace(',','').astype('int64')
print(df.shape)

# 데이터 추가, str -> float
# df2 = read_csv('c:/data/test/삼성전자0115.csv', encoding='cp949', index_col=0, header=0, thousands=',')
df2 = pd.read_csv(io.BytesIO(myfile['삼성전자0115.csv']), encoding='cp949', index_col=0, header=0, thousands=',')
df2 = df2.dropna()
df2 = df2.drop(['전일비','Unnamed: 6'], axis=1)

# 중복 데이터 제거
print(df.shape)
df = df.drop(['2021-01-13'])
df2 = df2.drop(df2.index[3:])
print(df2.shape)


# 액면분할 이전 데이터 주가변환
df = df[::-1]
df2 = df2[::-1]
df.loc[:'2018-05-04','시가':'종가'] = (df.loc[:'2018-05-04','시가':'종가'])/50.
df.loc[:'2018-05-04','거래량'] = (df.loc[:'2018-05-04','거래량'])*50.
df.loc[:'2018-05-04','개인':'프로그램'] = (df.loc[:'2018-05-04','개인':'프로그램'])*50.

# 데이터 병합
df = pd.concat([df, df2], axis=0)
print(df.shape)
print(df.tail())

# 타겟(y값) 설정
dataset_y = df.iloc[:,0]
df['Target'] = dataset_y

# 불필요한 특성 제거
# df = df.drop(['거래량','신용비','외인비'], axis=1)
# df = df.drop(['개인','기관','외인(수량)','외국계','프로그램'], axis=1)
print(df.shape)

# csv -> npy 후 저장
data = df.values
print(data)
# np.save('c:/data/test/samsung_jusik2.npy', arr=data)
np.save('/content/drive/My Drive/data/test/samsung_jusik.npy', arr=data)



(2400, 14)
(2397, 14)
(2397, 14)
(3, 14)
(2399, 14)
                 시가       고가       저가  ...        외국계       프로그램    외인비
일자                                     ...                             
2021-01-11  90000.0  96800.0  89500.0  ... -4979740.0 -6795684.0  55.59
2021-01-12  90300.0  91400.0  87800.0  ... -2093652.0 -4498684.0  55.56
2021/01/13  89800.0  91200.0  89100.0  ... -2774590.0 -2190774.0  55.52
2021/01/14  88700.0  90000.0  88700.0  ...  2193784.0 -1091335.0  55.57
2021/01/15  89800.0  91800.0  88000.0  ...  -261904.0 -3522801.0  55.57

[5 rows x 14 columns]
(2399, 15)
[[ 1.782000e+04  1.784000e+04  1.732000e+04 ...  3.567500e+05
   5.088000e+01  1.782000e+04]
 [ 1.706000e+04  1.750000e+04  1.706000e+04 ... -7.192500e+05
   5.090000e+01  1.706000e+04]
 [ 1.778000e+04  1.832000e+04  1.772000e+04 ...  3.235600e+06
   5.093000e+01  1.778000e+04]
 ...
 [ 8.980000e+04  9.120000e+04  8.910000e+04 ... -2.190774e+06
   5.552000e+01  8.980000e+04]
 [ 8.870000e+04  9.000000e+04  8.

In [ ]:
import numpy as np
import pandas as pd
import io
# from pandas import read_csv
# df = read_csv('c:/data/test/삼성전자.csv', index_col=0, header=0)
df = pd.read_csv(io.BytesIO(myfile['KODEX 코스닥150 선물인버스.csv']), index_col=0, header=0, encoding='cp949', thousands=',')


print(df.shape) # (2400, 14)
#Index(['시가', '고가', '저가', '종가', '등락률', '거래량', '금액(백만)', '신용비', '개인', '기관',
#       '외인(수량)', '외국계', '프로그램', '외인비']

# # str -> int, 불필요한 행 제거
df = df.drop(['2018/05/03','2018/05/02','2018/04/30'])
# df['시가'] =df['시가'].str.replace(',','').astype('int64')
# df['고가'] =df['고가'].str.replace(',','').astype('int64')
# df['저가'] =df['저가'].str.replace(',','').astype('int64')
# df['종가'] =df['종가'].str.replace(',','').astype('int64')
# df['거래량'] =df['거래량'].str.replace(',','').astype('int64')
# df['금액(백만)'] =df['금액(백만)'].str.replace(',','').astype('int64')
# df['개인'] =df['개인'].str.replace(',','').astype('int64')
# df['기관'] =df['기관'].str.replace(',','').astype('int64')
# df['외인(수량)'] =df['외인(수량)'].str.replace(',','').astype('int64')
# df['외국계'] =df['외국계'].str.replace(',','').astype('int64')
# df['프로그램'] =df['프로그램'].str.replace(',','').astype('int64')
# print(df.shape)

# 데이터 추가, str -> float
# df2 = read_csv('c:/data/test/삼성전자0115.csv', encoding='cp949', index_col=0, header=0, thousands=',')
# df2 = pd.read_csv(io.BytesIO(myfile['삼성전자0115.csv']), encoding='cp949', index_col=0, header=0, thousands=',')
df = df.dropna()
df = df.drop(['전일비','Unnamed: 6'], axis=1)

# # 중복 데이터 제거
# print(df.shape)
# df = df.drop(['2021-01-13'])
# df2 = df2.drop(df2.index[3:])
# print(df2.shape)
df = df[::-1]

# # 액면분할 이전 데이터 주가변환
# df = df[::-1]
# df2 = df2[::-1]
# df.loc[:'2018-05-04','시가':'종가'] = (df.loc[:'2018-05-04','시가':'종가'])/50.
# df.loc[:'2018-05-04','거래량'] = (df.loc[:'2018-05-04','거래량'])*50.
# df.loc[:'2018-05-04','개인':'프로그램'] = (df.loc[:'2018-05-04','개인':'프로그램'])*50.

# # 데이터 병합
# df = pd.concat([df, df2], axis=0)
# print(df.shape)
# print(df.tail())

# # 타겟(y값) 설정
# dataset_y = df.iloc[:,0]
# df['Target'] = dataset_y

# 불필요한 특성 제거
# df = df.drop(['거래량','신용비','외인비'], axis=1)
# df = df.drop(['개인','기관','외인(수량)','외국계','프로그램'], axis=1)
print(df.shape)

# csv -> npy 후 저장
data = df.values
print(data)
# np.save('c:/data/test/samsung_jusik2.npy', arr=data)
np.save('/content/drive/My Drive/data/test/kodex_jusik.npy', arr=data)


(1088, 16)
(1085, 14)
[[1.0000e+04 1.0000e+04 9.8650e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [9.8900e+03 1.0025e+04 9.8650e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.0020e+04 1.0025e+04 9.9450e+03 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [4.3700e+03 4.4100e+03 4.3350e+03 ... 0.0000e+00 0.0000e+00 4.4000e+00]
 [4.4000e+03 4.4400e+03 4.3800e+03 ... 0.0000e+00 0.0000e+00 4.5400e+00]
 [4.4200e+03 4.5450e+03 4.4050e+03 ... 0.0000e+00 0.0000e+00 4.5200e+00]]


In [ ]:
import numpy as np
import pandas as pd

# 1. 데이터
data1 = np.load('/content/drive/My Drive/data/test/samsung_jusik.npy')
data2 = np.load('/content/drive/My Drive/data/test/kodex_jusik.npy')

x1 = data1[1314:,:-1]
x2 = data2
y = data1[1314:,-1]
print(x1.shape, x2.shape)
print(y.shape)

from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()
scaler1.fit(x1)
x1 = scaler1.transform(x1)

scaler2 = MinMaxScaler()
scaler2.fit(x2)
x2 = scaler2.transform(x2)

size = 6

def split_x(seq, size):
    aaa = []
    for i in range(len(seq)-size+1):
        subset = seq[i : (i+size)]
        aaa.append([item for item in subset])
    print(type(aaa))
    return np.array(aaa)
x1_data = split_x(x1, size)
x2_data = split_x(x2, size)

x1 = x1_data[:-2,:,:]
x2 = x2_data[:-2,:,:]


y = y[7:]
print(x1.shape, x2.shape)
print(y.shape)


from sklearn.model_selection import train_test_split
x1_train, x1_test, x2_train, x2_test, y_train, y_test = train_test_split(x1, x2, y, train_size=0.8, shuffle=True, random_state=66)


#2. 모델구성
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, LSTM, Dropout, Conv1D, Flatten, MaxPooling1D, GRU, SimpleRNN
# inputs = Input(shape=(6, x.shape[2]))
# dense1 = Conv1D(1000, 2, padding='same', activation='relu')(inputs)
# dense1 = MaxPooling1D(pool_size=2)(dense1)
# dense1 = Conv1D(500, 2, activation='relu')(dense1)
# dense1 = Conv1D(400, 2,activation='relu')(dense1)
# dense1 = Flatten()(dense1)


# 모델 1
input1 = Input(shape=(6,x1.shape[2]))
dense1 = LSTM(512)(input1)
dense1 = Dense(256)(dense1)
dense1 = Dense(128)(dense1)
dense1 = Dense(64)(dense1)
dense1 = Dense(32)(dense1)
dense1 = Dense(16)(dense1)
dense1 = Dense(8)(dense1)
dense1 = Dense(4)(dense1)
dense1 = Dense(2)(dense1)


# 모델 2
input2 = Input(shape=(6,x2.shape[2]))
dense2 = LSTM(512)(input2)
dense2 = Dense(256)(dense2)
dense2 = Dense(128)(dense2)
dense2 = Dense(64)(dense2)
dense2 = Dense(32)(dense2)
dense2 = Dense(16)(dense2)
dense2 = Dense(8)(dense2)
dense2 = Dense(4)(dense2)
dense2 = Dense(2)(dense2)


from tensorflow.keras.layers import concatenate, Concatenate
merge1 = concatenate([dense1, dense2])
middle1 = Dense(128)(merge1)
middle1 = Dense(64)(middle1)
middle1 = Dense(32)(middle1)
middle1 = Dense(16)(middle1)
middle1 = Dense(8)(middle1)
middle1 = Dense(4)(middle1)
middle1 = Dense(2)(middle1)
outputs = Dense(1)(middle1)





model = Model(inputs=[input1,input2], outputs=outputs)


#3. 컴파일, 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mae'])
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', patience=100, mode='auto')
modelpath= '/content/drive/My Drive/data/test/modelcheckpoint/samsung_checkpoint_{val_loss:.4f}.hdf5'
cp = ModelCheckpoint(modelpath, monitor='val_loss', save_best_only=True, mode='auto')
model.fit([x1_train,x2_train], y_train, batch_size=64, epochs=1000, validation_split=0.2, callbacks=[cp, es])

model.save('/content/drive/My Drive/data/test/samsung_model2.h5')

#4. 평가, 예측
loss, mae = model.evaluate([x1_test,x2_test], y_test, batch_size=64)
y_predict = model.predict([x1_test, x2_test])
print("loss, mae : ", loss, mae)

from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_predict)
print("R2 : ", r2)

x1_pred = x1_data[-2:,:,:]
x2_pred = x2_data[-2:,:,:]
x1_pred = x1_pred.reshape(x1_pred.shape[0],6,x1_pred.shape[-1])
x2_pred = x2_pred.reshape(x2_pred.shape[0],6,x2_pred.shape[-1])
y_predict = model.predict([x1_pred,x2_pred])

print(y_predict)


print("월요일 삼성 시가 : ", int(np.round(y_predict[0])), "원")
print("화요일 삼성 시가 : ", int(np.round(y_predict[1])), "원")


(1085, 14) (1085, 14)
(1085,)
<class 'list'>
<class 'list'>
(1078, 6, 14) (1078, 6, 14)
(1078,)
Epoch 1/1000
11/11 [==============================] - 4s 80ms/step - loss: 2328565418.6667 - mae: 47372.1732 - val_loss: 1798982272.0000 - val_mae: 41517.8711
Epoch 2/1000
11/11 [==============================] - 0s 13ms/step - loss: 1135164842.6667 - mae: 30567.3029 - val_loss: 76268536.0000 - val_mae: 6278.5854
Epoch 3/1000
11/11 [==============================] - 0s 13ms/step - loss: 185638202.6667 - mae: 10962.1573 - val_loss: 75441528.0000 - val_mae: 6180.7471
Epoch 4/1000
11/11 [==============================] - 0s 13ms/step - loss: 130507240.6667 - mae: 8719.9219 - val_loss: 127003040.0000 - val_mae: 8813.9824
Epoch 5/1000
11/11 [==============================] - 0s 12ms/step - loss: 102351230.0000 - mae: 8017.9003 - val_loss: 85704088.0000 - val_mae: 6994.5605
Epoch 6/1000
11/11 [==============================] - 0s 11ms/step - loss: 78204488.6667 - mae: 6251.8581 - val_loss: 7498248

In [ ]:
import numpy as np
import pandas as pd

# 1. 데이터
data1 = np.load('/content/drive/My Drive/data/test/samsung_jusik.npy')
data2 = np.load('/content/drive/My Drive/data/test/kodex_jusik.npy')

x1 = data1[1314:,:-1]
x2 = data2
y = data1[1314:,-1]
print(x1.shape, x2.shape)
print(y.shape)

from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()
scaler1.fit(x1)
x1 = scaler1.transform(x1)

scaler2 = MinMaxScaler()
scaler2.fit(x2)
x2 = scaler2.transform(x2)

size = 6

def split_x(seq, size):
    aaa = []
    for i in range(len(seq)-size+1):
        subset = seq[i : (i+size)]
        aaa.append([item for item in subset])
    print(type(aaa))
    return np.array(aaa)
x1_data = split_x(x1, size)
x2_data = split_x(x2, size)

x1 = x1_data[:-2,:,:]
x2 = x2_data[:-2,:,:]


y = y[7:]
print(x1.shape, x2.shape)
print(y.shape)


from sklearn.model_selection import train_test_split
x1_train, x1_test, x2_train, x2_test, y_train, y_test = train_test_split(x1, x2, y, train_size=0.8, shuffle=True, random_state=66)


#2. 모델구성
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, LSTM, Dropout, Conv1D, Flatten, MaxPooling1D, GRU, SimpleRNN

# # 모델 1
# input1 = Input(shape=(6,x1.shape[2]))
# dense1 = LSTM(512)(input1)
# dense1 = Dense(256)(dense1)
# dense1 = Dense(128)(dense1)
# dense1 = Dense(64)(dense1)
# dense1 = Dense(32)(dense1)
# dense1 = Dense(16)(dense1)
# dense1 = Dense(8)(dense1)
# dense1 = Dense(4)(dense1)
# dense1 = Dense(2)(dense1)


# # 모델 2
# input2 = Input(shape=(6,x2.shape[2]))
# dense2 = LSTM(512)(input2)
# dense2 = Dense(256)(dense2)
# dense2 = Dense(128)(dense2)
# dense2 = Dense(64)(dense2)
# dense2 = Dense(32)(dense2)
# dense2 = Dense(16)(dense2)
# dense2 = Dense(8)(dense2)
# dense2 = Dense(4)(dense2)
# dense2 = Dense(2)(dense2)


# from tensorflow.keras.layers import concatenate, Concatenate
# merge1 = concatenate([dense1, dense2])
# middle1 = Dense(128)(merge1)
# middle1 = Dense(64)(middle1)
# middle1 = Dense(32)(middle1)
# middle1 = Dense(16)(middle1)
# middle1 = Dense(8)(middle1)
# middle1 = Dense(4)(middle1)
# middle1 = Dense(2)(middle1)
# outputs = Dense(1)(middle1)





# model = Model(inputs=[input1,input2], outputs=outputs)


# #3. 컴파일, 훈련
# model.compile(loss='mse', optimizer='adam', metrics=['mae'])
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# es = EarlyStopping(monitor='val_loss', patience=100, mode='auto')
# modelpath= '/content/drive/My Drive/data/test/modelcheckpoint/samsung_checkpoint_{val_loss:.4f}.hdf5'
# cp = ModelCheckpoint(modelpath, monitor='val_loss', save_best_only=True, mode='auto')
# model.fit([x1_train,x2_train], y_train, batch_size=64, epochs=1000, validation_split=0.2, callbacks=[cp, es])

# model = load_model('/content/drive/My Drive/data/test/samsung_model2.h5')
model = load_model('/content/drive/My Drive/data/test/modelcheckpoint/samsung_checkpoint_2285252.2500.hdf5')


#4. 평가, 예측
loss, mae = model.evaluate([x1_test,x2_test], y_test, batch_size=64)
y_predict = model.predict([x1_test, x2_test])
print("loss, mae : ", loss, mae)

from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE : ", RMSE(y_test, y_predict))

from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_predict)
print("R2 : ", r2)

x1_pred = x1_data[-2:,:,:]
x2_pred = x2_data[-2:,:,:]
x1_pred = x1_pred.reshape(x1_pred.shape[0],6,x1_pred.shape[-1])
x2_pred = x2_pred.reshape(x2_pred.shape[0],6,x2_pred.shape[-1])
y_predict = model.predict([x1_pred,x2_pred])

print(y_predict)


print("월요일 삼성 시가 : ", int(np.round(y_predict[0])), "원")
print("화요일 삼성 시가 : ", int(np.round(y_predict[1])), "원")

# 로드 모델
#loss, mae :  2541032.0 1262.46630859375
# RMSE :  1594.061363025406
# R2 :  0.9729504677582311
# [[91914.98]
#  [88692.26]]
# 월요일 삼성 시가 :  91915 원
# 화요일 삼성 시가 :  88692 원

# check
# loss, mae :  2288014.0 1166.26708984375
# RMSE :  1512.6178212770928
# R2 :  0.9756438794913104
# [[90409.25]
#  [88900.08]]
# 월요일 삼성 시가 :  90409 원
# 화요일 삼성 시가 :  88900 원


(1085, 14) (1085, 14)
(1085,)
<class 'list'>
<class 'list'>
(1078, 6, 14) (1078, 6, 14)
(1078,)
4/4 [==============================] - 1s 6ms/step - loss: 2288014.0000 - mae: 1166.2671
loss, mae :  2288014.0 1166.26708984375
RMSE :  1512.6178212770928
R2 :  0.9756438794913104
[[90409.25]
 [88900.08]]
월요일 삼성 시가 :  90409 원
화요일 삼성 시가 :  88900 원
